In [15]:
import os
import pandas as pd
import csv
import numpy as np

## Gathering Results

### Functions

In [16]:
# Function to remove extra column from bug
def removeExtraColumns(addresses):
    for address in addresses:
        df = pd.read_csv(address)
        if df.shape[1] > 2:
            df = df.drop(df.columns[2], axis=1)
            df.to_csv(address, index=False)

# Function to remove extra column from bug
def removeRedundantTitle(addresses):
    for address in addresses:
        frame = pd.read_csv(address)
        listOfNames = [frame.columns[0]]
        listOfNames += list(frame[frame.columns[0]])
        # print(listOfNames)

        tempList = []
        for name in listOfNames:
            if name[0] == "B":
                tempList.append(name.replace("BLR", ""))
            if name[0] == "R":
                tempList.append(name.replace("RF", ""))
            if name[0] == "N":
                tempList.append(name.replace("NN", ""))
            if name[0] == "C":
                tempList.append(name.replace("CoTraining", ""))

        # print(tempList)
        
        with open(address, 'r') as file:
            csv = file.read()

            for name in listOfNames:
                csv = csv.replace(name, tempList[0]); tempList.pop(0)

        with open(address, 'w') as file:
            file.write(csv)

In [17]:
def getCSVAddresses(rootFolder):
    listOfAddresses = []
    for model in list(os.listdir(rootFolder)):
        model = model + "/"
        csvFile = rootFolder + model + "ConfusionMatrices.csv"
        listOfAddresses.append(csvFile)
    return listOfAddresses

def getCompleteList(addresses):
    bigListOfOtherLists = []

    for address in addresses:
        # To get Model Name
        fileName = "/ConfusionMatrices.csv"
        temp = address.replace(fileName, "")
        indices = [i for i, char in enumerate(temp) if char == "/"]
        modelName = temp[indices[-1]+1:]

        tempList = []
        with open(address, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                tempDict = {}
                tempDict["Test Name"] = modelName + ": " + row[0]  
                tempDict["AverageConfusionMatrix"] = row[1]

                perFoldAddress = temp + "/Per Fold Results/" + row[0] + ".csv"

                with open(perFoldAddress, newline='') as perFoldFile:
                    foldReader = csv.reader(perFoldFile)
                    
                    for foldRows in foldReader:
                        if foldRows[0] != "foldNumber":
                            foldNumber = foldRows[0]
                            foldMatrix = foldRows[-1].replace("\n ", "")
                            foldMatrix = foldMatrix[1:]; foldMatrix = foldMatrix[:-1]

                            tempDict["Fold" + foldNumber] = foldMatrix

                tempList.append(tempDict)

        bigListOfOtherLists.append(tempList)
    
    finalList = []
    [finalList.extend(sublist) for sublist in bigListOfOtherLists]
    return finalList

### Gather RECOLA Results

In [18]:
folderLocation = "Datasets/RECOLA/" # Root Folder
fullTests = folderLocation + "Supervised Models/" # Supervised Results
semiTests = folderLocation + "Semi-Supervised Models/" # Semi-Supervised Results

# List of addresses
listOfAddresses = getCSVAddresses(fullTests) + getCSVAddresses(semiTests)

# removeExtraColumns(listOfAddresses)

# CSV of test and matrices
listOfAll = getCompleteList(listOfAddresses)

RECOLAConfusionMatrices = pd.DataFrame(listOfAll)

display(RECOLAConfusionMatrices)

,Test Name,AverageConfusionMatrix,Fold1,Fold2,Fold3,Fold4,Fold5,Fold6,Fold7,Fold8,Fold9
0,Binary Logistic Regression: AudioArousal,[193.44444444 99.66666667][ 95.22222222 197.7...,[155 154][ 74 205],[163 93][103 229],[252 78][128 129],[257 69][ 96 165],[186 68][104 228],[211 71][153 151],[107 158][ 43 277],[273 59][111 141],[137 147][ 45 255]
1,Binary Logistic Regression: AudioValence,[151. 142.11111111][114.22222222 178.7...,[128 125][115 220],[166 175][120 127],[ 74 221][ 47 245],[329 24][188 46],[ 63 204][ 42 277],[134 232][ 98 122],[162 108][105 210],[257 83][123 121],[ 46 107][190 241]
2,Binary Logistic Regression: VisualArousal,[141.77777778 151.33333333][103.77777778 189.2...,[139 170][ 90 189],[163 93][137 195],[ 65 265][ 23 234],[108 218][ 58 203],[193 61][249 83],[228 54][151 153],[141 124][124 196],[155 177][ 61 191],[ 84 200][ 41 259]
3,Binary Logistic Regression: VisualValence,[114.44444444 178.66666667][130.77777778 162.2...,[131 122][145 190],[238 103][188 59],[ 32 263][ 18 274],[136 217][ 84 150],[215 52][237 82],[154 212][104 116],[105 165][165 150],[ 5 335][ 1 243],[ 14 139][235 196]
4,Binary Logistic Regression: PhysiologyArousal,[177.55555556 115.55555556][112. 181.],[237 72][160 119],[208 48][202 130],[ 92 238][ 38 219],[256 70][160 101],[ 47 207][ 23 309],[192 90][109 195],[167 98][ 91 229],[228 104][153 99],[171 113][ 72 228]
...,...,...,...,...,...,...,...,...,...,...,...
307,Tri-Training: AllValenceRF8,[155.11111111 138. ][103.55555556 189.4...,[165 88][124 211],[173 168][ 98 149],[163 132][118 174],[129 224][ 41 193],[172 95][117 202],[228 138][ 75 145],[195 75][111 204],[111 229][ 62 182],[ 60 93][186 245]
308,Tri-Training: AllValenceRF12,[147.33333333 145.77777778][103.33333333 189.6...,[149 104][123 212],[187 154][123 124],[152 143][ 80 212],[127 226][ 58 176],[166 101][124 195],[228 138][ 67 153],[174 96][104 211],[ 85 255][ 63 181],[ 58 95][188 243]
309,Tri-Training: AllValenceNN4,[166. 127.11111111][126.33333333 166.6...,[151 102][174 161],[252 89][150 97],[ 78 217][ 58 234],[129 224][ 36 198],[114 153][ 70 249],[254 112][144 76],[188 82][118 197],[315 25][225 19],[ 13 140][162 269]
310,Tri-Training: AllValenceNN8,[136.33333333 156.77777778][106.77777778 186.2...,[168 85][166 169],[194 147][122 125],[108 187][ 77 215],[171 182][ 49 185],[ 55 212][ 24 295],[139 227][121 99],[184 86][129 186],[196 144][129 115],[ 12 141][144 287]


### Gather AGAIN Results

In [19]:
folderLocation = "Datasets/AGAIN/" # Root Folder
fullTests = folderLocation + "Supervised Models/" # Supervised Results
semiTests = folderLocation + "Semi-Supervised Models/" # Semi-Supervised Results
games = ["TopDown/", "Shootout/", "Heist!/"]; allTests = []
for game in games: allTests.append(fullTests + game); allTests.append(semiTests + game)

# List of addresses
listOfAddresses = []
for test in allTests:
    listOfAddresses += getCSVAddresses(test)

# removeExtraColumns(listOfAddresses)
# removeRedundantTitle(listOfAddresses)

# CSV of test and matrices
listOfAll = getCompleteList(listOfAddresses)
AGAINConfusionMatrices = pd.DataFrame(listOfAll)
display(AGAINConfusionMatrices)

,Test Name,AverageConfusionMatrix,Fold1,Fold2,Fold3,Fold4,Fold5,Fold6,Fold7,Fold8,Fold9,Fold10
0,Binary Logistic Regression: TopDown,[138.9 41.6][ 44.4 142.1],[120 71][ 49 135],[143 37][ 49 147],[125 55][ 48 146],[134 39][ 53 135],[139 29][ 31 162],[123 55][ 57 126],[152 26][ 36 146],[134 43][ 41 153],[168 29][ 35 138],[151 32][ 45 133]
1,Neural Network: TopDown,[133.9 46.6][ 46.7 139.8],[117 74][ 56 128],[131 49][ 54 142],[129 51][ 61 133],[130 43][ 48 140],[131 37][ 27 166],[121 57][ 60 123],[147 31][ 34 148],[141 36][ 46 148],[153 44][ 37 136],[139 44][ 44 134]
2,Random Forest: TopDown,[134.2 46.3][ 42.5 144. ],[115 76][ 50 134],[136 44][ 44 152],[122 58][ 41 153],[132 41][ 49 139],[134 34][ 29 164],[119 59][ 62 121],[148 30][ 35 147],[135 42][ 35 159],[153 44][ 34 139],[148 35][ 46 132]
3,Assemble: TopDownBLR4,[136.1 44.4][ 42.3 144.2],[116 75][ 58 126],[143 37][ 45 151],[124 56][ 51 143],[135 38][ 36 152],[140 28][ 34 159],[124 54][ 66 117],[149 29][ 31 151],[128 49][ 35 159],[161 36][ 33 140],[141 42][ 34 144]
4,Assemble: TopDownBLR8,[137.5 43. ][ 43.3 143.2],[115 76][ 54 130],[142 38][ 46 150],[122 58][ 46 148],[136 37][ 51 137],[139 29][ 34 159],[119 59][ 62 121],[149 29][ 29 153],[135 42][ 42 152],[167 30][ 26 147],[151 32][ 43 135]
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Tri-Training: Heist!RF8,[115.6 42.3][ 46.9 116.8],[117 44][ 45 110],[125 41][ 40 110],[ 92 55][ 61 108],[124 17][ 48 126],[120 33][ 36 125],[119 51][ 38 120],[116 44][ 53 115],[107 65][ 35 122],[133 34][ 51 109],[103 39][ 62 123]
113,Tri-Training: Heist!RF12,[114.9 43. ][ 47.8 115.9],[118 43][ 44 111],[117 49][ 37 113],[ 95 52][ 65 104],[117 24][ 47 127],[126 27][ 39 122],[119 51][ 40 118],[109 51][ 52 116],[111 61][ 46 111],[132 35][ 51 109],[105 37][ 57 128]
114,Tri-Training: Heist!NN4,[100.8 57.1][ 54.3 109.4],[105 56][ 44 111],[ 99 67][ 45 105],[ 82 65][ 60 109],[105 36][ 60 114],[ 97 56][ 59 102],[107 63][ 48 110],[103 57][ 67 101],[ 95 77][ 43 114],[123 44][ 51 109],[ 92 50][ 66 119]
115,Tri-Training: Heist!NN8,[110.6 47.3][ 49.1 114.6],[117 44][ 50 105],[108 58][ 42 108],[ 97 50][ 68 101],[112 29][ 62 112],[105 48][ 31 130],[121 49][ 39 119],[115 45][ 59 109],[101 71][ 47 110],[130 37][ 39 121],[100 42][ 54 131]


## Calculating Results

In [20]:
def calculatePerFoldResults(row):
    accuracies = []
    precisions = []
    recalls = []
    f1Scores = []

    for fold in row:
        tempString = ""
        for char in fold:
            tempString += char
            if char == "[": tempString += " "
        matrixString = tempString.replace("[", "").replace("]", "").split()  
        trueConfMatrix = [[matrixString[0], matrixString[1]], [matrixString[2], matrixString[3]]]     

        # Split by quadrant
        TP, FP, FN, TN = float(trueConfMatrix[0][0]), float(trueConfMatrix[0][1]), float(trueConfMatrix[1][0]), float(trueConfMatrix[1][1])
        accuracies.append((TP + TN)/(TP + FP + FN + TN)) if (TP + FP + FN + TN) != 0 else accuracies.append(0)
        precisions.append((TP)/(TP + FP)) if (TP + FP) != 0 else precisions.append(0)
        recalls.append((TP)/(TP + FN)) if (TP + FN) != 0 else recalls.append(0)
        f1Scores.append(2 * ((precisions[-1]*recalls[-1])/(precisions[-1]+recalls[-1]))) if (precisions[-1]+recalls[-1]) != 0 else f1Scores.append(0)

    return np.std(accuracies), np.std(precisions), np.std(recalls), np.std(f1Scores)

def calculateResults(frame, fileName):
    results = []    
    for _, row in frame.iterrows():
        testName = row["Test Name"]
        matrixString = row["AverageConfusionMatrix"]

        tempString = ""
        for char in matrixString:
            tempString += char
            if char == "[": tempString += " "
        matrixString = tempString.replace("[", "").replace("]", "").split()        

        trueConfMatrix = [[matrixString[0], matrixString[1]], [matrixString[2], matrixString[3]]]   
        TP, FP, FN, TN = float(trueConfMatrix[0][0]), float(trueConfMatrix[0][1]), float(trueConfMatrix[1][0]), float(trueConfMatrix[1][1])

        accuracy = (TP + TN)/(TP + FP + FN + TN) if (TP + FP + FN + TN) != 0 else 0
        precision = (TP)/(TP + FP) if (TP + FP) != 0 else 0
        recall = (TP)/(TP + FN) if (TP + FN) != 0 else 0
        f1Score = 2 * ((precision*recall)/(precision+recall)) if (precision+recall) != 0 else 0
        
        stanDevAccuracy, stanDevPrecision, stanDevRecall, stanDevF1Score = calculatePerFoldResults(row[2:])
        results.append({"Test Name":testName, 
                        "Accuracy":accuracy, "Precision":precision, "Recall":recall, "F1 Score":f1Score, 
                        "Accuracy (Standard Deviation)": stanDevAccuracy, "Precision (Standard Deviation)": stanDevPrecision, "Recall (Standard Deviation)": stanDevRecall, "F1 Score (Standard Deviation)": stanDevF1Score})
    
    finalFrame = pd.DataFrame(results)
    finalFrame.to_csv(fileName, index=False)
    return finalFrame

In [ ]:
folderName = "Results/"
recolaFinal = calculateResults(RECOLAConfusionMatrices, folderName + "RECOLA_Results.csv")
againFinal = calculateResults(AGAINConfusionMatrices, folderName + "AGAIN_Results.csv")